In [1]:
import numpy as np

# generate a random matrix
dna = np.random.choice(list('ACTG'),(7,21),replace=True)


DNA_1 = {'G': { 'G':1, 'C':0, 'A':0, 'T':0 },
         'C': { 'G':0, 'C':1, 'A':0, 'T':0 },
         'A': { 'G':0, 'C':0, 'A':1, 'T':0 },
         'T': { 'G':0, 'C':0, 'A':0, 'T':1 }}


# generate optimal pairwise sequence
def sequenceAlign(seqA, seqB, simMatrix=DNA_1, insert=8, extend=4):

  numI = len(seqA) + 1
  numJ = len(seqB) + 1

  scoreMatrix = [[0] * numJ for x in range(numI)]
  routeMatrix = [[0] * numJ for x in range(numI)]
  
  for i in range(1, numI):
    routeMatrix[i][0] = 1
  
  for j in range(1, numJ):
    routeMatrix[0][j] = 2
  
  for i in range(1, numI):
    for j in range(1, numJ):
    
      penalty1 = insert
      penalty2 = insert
      
      if routeMatrix[i-1][j] == 1:
        penalty1 = extend
        
      elif routeMatrix[i][j-1] == 2:
        penalty2 = extend
        
      similarity = simMatrix[ seqA[i-1] ][ seqB[j-1] ]
      
      paths = [scoreMatrix[i-1][j-1] + similarity, # Route 0
               scoreMatrix[i-1][j] - penalty1, # Route 1
               scoreMatrix[i][j-1] - penalty2] # Route 2                     
      
      best = max(paths)
      route = paths.index(best)           

      scoreMatrix[i][j] = best
      routeMatrix[i][j] = route
      
  alignA = []
  alignB = []
  
  i = numI-1
  j = numJ-1
  score = scoreMatrix[i][j]

    
  while i > 0 or j > 0:
    route = routeMatrix[i][j]    

    if route == 0: # Diagonal
      alignA.append( seqA[i-1] )
      alignB.append( seqB[j-1] )
      i -= 1
      j -= 1

    elif route == 1: # Gap in seqB
      alignA.append( seqA[i-1] )
      alignB.append( '-' )
      i -= 1      

    elif route == 2: # Gap in seqA
      alignA.append( '-' )
      alignB.append( seqB[j-1] ) 
      j -= 1
  
  alignA.reverse()
  alignB.reverse()
  alignA = ''.join(alignA)
  alignB = ''.join(alignB)

  return score, alignA, alignB


# use substitution matrix to calculate similarity score for two sequences
def calcSeqSimilarity(seqA, seqB, simMatrix):

  numPlaces = min(len(seqA), len(seqB))
  
  totalScore = 0.0
  
  for i in range(numPlaces):
    
    residueA = seqA[i]
    residueB = seqB[i]
  
    totalScore += simMatrix[residueA][residueB]

  return totalScore


# calculate distance by calculating maximum possible score then subtracting actual score
def getDistanceMatrix(seqs, simMatrix):

  n = len(seqs)
  matrix = [[0.0] * n for x in range(n)]
  maxScores = [calcSeqSimilarity(x, x, simMatrix) for x in seqs]

  for i in range(n-1):
    seqA = seqs[i]
  
    for j in range(i+1,n):
      seqB = seqs[j]
      
      score, alignA, alignB = sequenceAlign(seqA, seqB, simMatrix)
      maxScore = max(maxScores[i],maxScores[j])
      dist = maxScore - score
      
      matrix[i][j] = dist
      matrix[j][i] = dist

  return matrix

distMatrix = getDistanceMatrix(dna, DNA_1)
print(distMatrix)

[[0.0, 15.0, 16.0, 16.0, 15.0, 15.0, 19.0], [15.0, 0.0, 16.0, 16.0, 15.0, 16.0, 18.0], [16.0, 16.0, 0.0, 18.0, 17.0, 14.0, 13.0], [16.0, 16.0, 18.0, 0.0, 13.0, 19.0, 18.0], [15.0, 15.0, 17.0, 13.0, 0.0, 14.0, 14.0], [15.0, 16.0, 14.0, 19.0, 14.0, 0.0, 12.0], [19.0, 18.0, 13.0, 18.0, 14.0, 12.0, 0.0]]
